If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install transformers[sentencepiece]
! pip install sacrebleu -U
! pip install datasets -U
! pip install transformers -U
! pip install unbabel-comet -U
! pip install rouge
! pip install accelerate -U
! pip install evaluate -U
! pip install bertscore -U
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers
import torch
print(torch.cuda.is_available())
print(transformers.__version__)
print(torch.__version__)
!python --version

True
4.44.2
2.4.0+cu121
Python 3.10.12


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
# from transformers.utils import send_example_telemetry

# send_example_telemetry("translation_notebook", framework="pytorch")

# Fine-tuning a model on a translation task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a translation task. We will use the [WMT dataset](http://www.statmt.org/wmt16/), a machine translation dataset composed from a collection of various sources, including news commentaries and parliament proceedings.

![Widget inference on a translation task](https://github.com/huggingface/notebooks/blob/main/examples/images/translation.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [ ]:
model_checkpoint = "facebook/m2m100_418M"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-en-ro`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) checkpoint.

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. We use the English/Romanian part of the WMT dataset here.

In [ ]:
from torch.utils.data import DataLoader, Dataset
import random
import json

def ReadFile(path):
    with open(path, encoding="utf-8") as file:
        return file.readlines()

def WriteFile(path, data):
    with open(path, "a", encoding="utf-8") as file:
        file.write(data)

def WriteJsonFile(path, data):
    with open(path, "a", encoding="utf-8") as file:
        file.write(json.dumps(data))

def filter_invalid_characters(sentence):
    arm_voc = ['և', ' ',  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '՞', '«', '»', ':', ',',
            'ա', 'բ', 'գ', 'դ', 'ե', 'զ', 'է','ը','թ','ժ','ի','լ','խ','ծ','կ','հ','ձ','ղ','ժ','մ','յ','ն','շ','ո','չ','պ','ջ','ռ','ս','վ','տ','ր','ց','ւ', 'փ', 'ք','օ','ֆ',
            'Ա', 'Բ', 'Գ', 'Դ', 'Ե', 'Զ', 'Է','Ը','Թ','Ժ','Ի','Լ','Խ','Ծ','Կ','Հ','Ձ','Ղ','Ժ','Շ','Յ','Ն','Շ','Ո','Չ','Պ','Ջ','Ռ','Ս','Վ','Տ','Ր','Ց','Ւ', 'Փ', 'Ք','Օ','Ֆ']
    eng_voc = [' ', '?', '"', '.', ',',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z']
    valid_characters = arm_voc + eng_voc
    filtered_sentence = ''
    for char in sentence:
        if char in valid_characters:
            filtered_sentence += char
    return filtered_sentence

def filter_sentences(sentences, valid_characters):
    return list(map(lambda s: filter_invalid_characters(s, valid_characters), sentences))


def LoadDataset(pathlistlang1, pathlistlang2, ind = -1):
    listofdicts = []
    if ind != -1:
        pathlistlang1 = [pathlistlang1[ind]]
        pathlistlang2 = [pathlistlang2[ind]]
    for pathlang1, pathlang2 in zip(pathlistlang1, pathlistlang2):
        with open(pathlang1, encoding="utf-8") as file1, open(pathlang2, encoding="utf-8") as file2:
            for line1, line2 in zip(file1, file2):
                line1 = filter_invalid_characters(line1.lower())
                line2 = filter_invalid_characters(line2.lower())
                listofdicts.append({"en": line1.rstrip(), "hyw": line2.rstrip()})
    return listofdicts

def split_data(data, train_percent, test_percent):
    # Shuffle the data to ensure randomness
    random.shuffle(data)

    # Calculate the number of items for the training and test sets
    train_size = int(len(data) * train_percent)
    test_size = int(len(data) * test_percent)

    # Split the data into training, validation, and test sets
    train_data = data[:train_size]
    test_data = data[train_size:train_size + test_size]
    validation_data = data[train_size + test_size:]

    return train_data, validation_data, test_data


def LoadDataSetDictFromPaths(pathlistlang1, pathlistlang2, train_percent = 0.7, test_percent = 0.15, split_data = False):
    if split_data:
      alldata = LoadDataset(pathlistlang1, pathlistlang2)
      train, validate, test = split_data(alldata, train_percent, test_percent)
    else:
      train = LoadDataset(pathlistlang1, pathlistlang2, 0)
      test = LoadDataset(pathlistlang1, pathlistlang2, 1)
      validate = LoadDataset(pathlistlang1, pathlistlang2, 2)
    return DatasetDict({
        "train": datasets.Dataset.from_dict({"translation": train}),
        "test":  datasets.Dataset.from_dict({"translation": test}),
        "val":   datasets.Dataset.from_dict({"translation": validate})
    })



In [ ]:
# from datasets import load_metric

# metric = load_metric("sacrebleu", trust_remote_code=True)

In [ ]:
from datasets import load_dataset, DatasetDict
import datasets
from transformers import MT5Tokenizer, M2M100Tokenizer

#dir = "/content/drive/MyDrive/Colab Notebooks/hyw-en-parallel-corpus/"
dir = "/content/drive/MyDrive/Colab Notebooks/models"
#dir = "../../hyw-en-parallel-corpus/"
pathlistlang1 = []#[f"{dir}bible/bible.en",  f"{dir}hamazkayin/hamazkayin.en",  f"{dir}hayernaysor/hayernaysor.en",  f"{dir}aalw/aalw.en",  f"{dir}wikipedia/wikipedia.en"]
pathlistlang2 = []#[f"{dir}bible/bible.hyw", f"{dir}hamazkayin/hamazkayin.hyw", f"{dir}hayernaysor/hayernaysor.hyw", f"{dir}aalw/aalw.hyw", f"{dir}wikipedia/wikipedia.hyw"]
for lang in ["en", "hyw"]:
  for val in ["train", "test", "val"]:
    if lang == "en":
      pathlistlang1.append(f"{dir}/{val}_{lang}.txt")
    else:
      pathlistlang2.append(f"{dir}/{val}_{lang}.txt")
raw_datasets = LoadDataSetDictFromPaths(pathlistlang1, pathlistlang2)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 37015
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 7931
    })
    val: Dataset({
        features: ['translation'],
        num_rows: 7933
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
raw_datasets["train"][37014]

{'translation': {'en': 'when you pass through the waters i will be with you and through the rivers they shall not overwhelm you when you walk through fire you shall not be burned and the flame shall not consume you.',
  'hyw': 'երբ ջուրերէն անցնիս ես քեզի հետ պիտի ըլլամ երբ գետերէն անցնիս պիտի չընկղմիս երբ կրակի մէջէն քալես պիտի չայրիս բոցը քեզ պիտի չբռնկեցնէ'}}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["val"])

,translation
0,"{'en': 'eleazar became the father of phinehas phinehas of abishua', 'hyw': 'եղիազար ծնաւ փենէհէսը ու փենէհէս ծնաւ աբիսուան'}"
1,"{'en': 'like wild asses in the desert they go out to their toil scavenging in the wasteland food for their young.', 'hyw': 'ահա անոնք անապատի վայրենի էշերուն պէս իրենց գործին կելլեն կանուխ որս փնտռելու կերթան. անապատը անոնց տղոցը հաց կու տայ'}"
2,"{'en': 'the audience members enjoyed and appreciated the warm atmosphere of the evening event where everyone expressed their wish that the activities of the club might be ongoing.', 'hyw': 'ներկաները ոչ միայն դարձան աւելի իմաստուն այլ միասնաբար վայելեցին ու գնահատեցին շատ հաելի ջերմ եւ ընկերային մթնոլորտը մաղթելով որ անխափան շարունակուի ակումբի գործունէութիւնը'}"
3,"{'en': 'through want and hard hunger they gnaw the dry and desolate ground', 'hyw': 'չքաւորութենէ ու անօթութենէ նիհարցեր էին դէպի անապատը մթին աւերակ ու վայրի տեղերը կը փախչէին'}"
4,"{'en': 'she welcomed the elected officials and expressed gratitude to hagop balian the late loucine malikian and seta biberian for serving at various times as principals of the school.', 'hyw': 'երախտագիտութիւն յայտնեց յակոբ պալեանի հանգուցեալ լուսին մալիքեանի եւ սեդա պիւպերեանի տարիներու ընթացքին տնօրէնութեան պաշտօնը վարած ըլլալնուն համար եւ ներկայացուց դպրոցի 39 տարիներու յառաջխաղացքը'}"


In [ ]:
# Write train, test, and validation into a file
# import json
# def WriteToFile(path, data, lang):
#     with open(path, "w", encoding="utf-8") as file:
#         for sentence in data:
#             a = sentence[lang] + "\n"
#             file.write(a)

# for lang in ["en", "hyw"]:
#   for datatype in ["train", "test", "val"]:
#     WriteToFile(f"/content/drive/MyDrive/Colab Notebooks/models/{datatype}_{lang}.txt", raw_datasets[datatype]["translation"], lang)

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
import evaluate
metric = evaluate.load("sacrebleu")

In [ ]:
metric

EvaluationModule(name: "sacrebleu", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'e

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer, M2M100Tokenizer

tokenizer = M2M100Tokenizer.from_pretrained(model_checkpoint)
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "hy"

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes [here](https://huggingface.co/facebook/mbart-large-cc25) if you are using this notebook on a different pairs of languages.

In [ ]:
tokenizer.pad_token_id

1

In [ ]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [128022, 65761, 4, 15911, 13331, 8931, 6226, 30, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[128022, 65761, 4, 15911, 13331, 8931, 6226, 30, 2], [128022, 36606, 117, 122000, 8931, 6226, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[128040, 65761, 4, 15911, 13331, 8931, 6226, 30, 2], [128040, 36606, 117, 122000, 8931, 6226, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "hyw"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors='pt', padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, return_tensors='pt', padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': tensor([[128022,   1095,     81,  18685,  33969,  13635,    667,  63503,  62843,
           1019,  64975,  95866, 118464,    673,     28,   1197, 125889,   1197,
             38,   1637,    432,  28119,     55,   1197,   3184,    432, 102117,
            117,    117,    220,   1435,   3184,   1019,   1197,  77432,    432,
          18685,  11956,    336,  25137,    192,  13635,    667,  63503,  62843,
             28,  18685,   2900,   6226,      5,      2],
        [128022,  91186,   8251,  13635, 124634,   1197,  47232,    432,   1197,
             38,   1637,   1019,  41386,   1197, 123788,    432,   3184,      5,
              2,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/37015 [00:00<?, ? examples/s]

Map:   0%|          | 0/7931 [00:00<?, ? examples/s]

Map:   0%|          | 0/7933 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_name = model_checkpoint.split("/")[-1]
a = f"{model_name}-finetuned-{source_lang}-to-{target_lang}"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)#(f"/content/drive/MyDrive/Colab Notebooks/models/{model_checkpoint}/{a}/")

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
# args = Seq2SeqTrainingArguments(
#     f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=1,
#     num_train_epochs=10,
#     fp16=False,
#     generation_max_length=max_target_length,
#     predict_with_generate=True,
#     #push_to_hub=True,
# )

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,  # Regularization via weight decay
    save_total_limit=3,  # Limit the number of saved checkpoints
    num_train_epochs=10,
    fp16=False,
    generation_max_length=max_target_length,
    logging_dir=f"./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/marian-finetuned-en-to-ro"` or `"huggingface/marian-finetuned-en-to-ro"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Replace -100s used for padding
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-processing for both predictions and labels
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Print one sample (first prediction and corresponding label)
    print("Sample Prediction:")
    print(decoded_preds[0])  # First prediction
    print("\nSample Reference:")
    print(decoded_labels[0])  # First reference

    # Compute the metrics
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Iterate over the result and handle lists
    for k, v in result.items():
        print(k, v)
        if isinstance(v, list):
            result[k] = round(np.mean(v), 4)
        else:
            result[k] = round(v, 4)

    # Compute the average length of predictions
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    return result


Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
import copy
from transformers import EarlyStoppingCallback

for param in model.parameters():
  param.data = param.data.contiguous()

# trainer = Seq2SeqTrainer(
#     model,
#     args,
#     train_dataset = tokenized_datasets["train"],
#     eval_dataset = tokenized_datasets["val"],
#     data_collator = data_collator,
#     tokenizer = tokenizer,
#     compute_metrics = compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
# )

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator = data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Early stopping
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.992400,1.998784
2,2.112500,2.004026
3,1.979100,1.949443
4,1.915300,1.931034


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instea

KeyboardInterrupt: 

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.save_model(f"/content/drive/MyDrive/Colab Notebooks/models/{model_checkpoint}/{args.output_dir}/")

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```

Evaluation

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/models/{model_checkpoint}/{args.output_dir}/")


In [ ]:
import torch
torch.cuda.empty_cache()

def Translate(input_sentence):
    input_ids = tokenizer(input_sentence, return_tensors="pt").input_ids
    #print(input_ids)
    outputs = model.generate(input_ids, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(Translate("I am going insane right now"))

def compute_meteor(hypothesis, reference):
        tokref = tokenizer.tokenize(reference)
        tokhyp = tokenizer.tokenize(hypothesis)
        meteor = meteor_score([tokref], tokhyp)
        return meteor

rouge = Rouge()
def compute_rouge(hypothesis, reference):
    # Compute ROUGE score
    scores = rouge.get_scores(hypothesis, reference)
    return scores

def compute_bleu(hypothesis, references):
    return corpus_bleu(hypothesis, references).score

def compute_sentence_bleu(hypothesis, references):
    return sentence_bleu(hypothesis, references).score

def compute_chrf(hypothesis, references):
    return corpus_chrf(hypothesis, references)

def compute_sentence_chrf(hypothesis, references):
    return sentence_chrf(hypothesis, references).score

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
dir = "/content/drive/MyDrive/Colab Notebooks/hyw-en-parallel-corpus"

from sacrebleu import corpus_bleu, corpus_chrf, sentence_bleu, sentence_chrf
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from tqdm import tqdm

In [ ]:
import multiprocessing.dummy as mp

unrelateddata = []
am = "american_citizen/american_citizen"
unrelateddata = LoadDataset([f"{dir}/{am}.{source_lang}"], [f"{dir}/{am}.{target_lang}"])


sources = []
references = []
hypothesis = []
sources = []
for sentence in tqdm(unrelateddata):
    #hyw_ids = Translate(sentence["en"])
    hyw = Translate(sentence["en"])
    #print(hyw)
    hypothesis.append(hyw)
    references.append(sentence["hyw"])
    sources.append(sentence["en"])

In [ ]:
print(sources[0])
print(len(sources[0]))

In [ ]:
def CalculateEvals(sources, hypothesises, references):
    import nltk
    nltk.download('wordnet')
    bleus = []
    chrfs = []
    rouges = []
    counter = 0
    meteos = []
    english = sources
    all = []
    bleus = compute_bleu(hypothesises, references)
    chrfs = compute_chrf(hypothesises, references)
    for source, reference, hypothesis in tqdm(zip(sources, references, hypothesises), "translating..."):
            en_hyw_translation = hypothesis
            #hypothesis = tokenizer.decode(en_hyw_translation[0], skip_special_tokens=True)
            bleu = compute_sentence_bleu(hypothesis, references)
            #bleus.append(bleu)
            chrf = compute_sentence_chrf(hypothesis, references)
            #chrfs.append(chrf)
            rouge = compute_rouge(en_hyw_translation, reference)[0]["rouge-l"]["f"]
            rouges.append(rouge)
            meteo = compute_meteor(en_hyw_translation, reference)
            meteos.append(meteo)
            all.append({
                "src": source,
                "hyp": en_hyw_translation,
                "ref": reference,
                "bleu": bleu,
                "chrf": chrf,
                "rouge-l": rouge,
                "meteo": meteo,
            })
    with open("/content/drive/MyDrive/Colab Notebooks/models/facebook/m2m-100_citizens_trans_eval2.json", 'w', encoding='utf-8') as output_file:
        json.dump(all, output_file, ensure_ascii=False, indent=4)
    return bleus, chrfs, rouges, meteos